<img style="float: left;padding: 1.3em" src="https://raw.githubusercontent.com/gw-odw/odw-2022/main/Tutorials/logo.png">  

#  Gravitational Wave Open Data Workshop #5

#### Tutorial A.1: Parameter estimation on GW190828_063405 using GWOSC data and LALInference software.

This tutorial provides step-by-step instructions to estimate the properties of the source of the gravitational events (GW) detected by the LIGO-Virgo-KAGRA (LVK) collaboration. The parameter estimation is performed with `LALInference`, a software library developed by the LVK collaboration to perform Bayesian inference of GW source properties for binary systems of black holes and/or neutron stars, and used during the three first observation runs. This tutorial can be adapted to analyse any event in the GWTC-2 & 3 catalogs. 

#### Content
The tutorial provides instruction to: 
- install `LALInference`
- download the GW strain 
- downlaod the PSD and calibration envelops
- read the configuration options
- start a `LALInference` run

#### Note
This tutorial is intended to be ran with a local `conda` installation on a mac or Linux computer, and does not support mybinder nor Google Colab.

It is adviced to use the official `IGWN` environment, see the [documentation](https://computing.docs.ligo.org/conda/) for details and instructions.

Instructions about setting up a `conda` environement are available on [this page](https://github.com/gw-odw/odw-2022/blob/main/setup.md) (option 3). 
The following packages must be installed in the conda environment: 
- `conda install --channel conda-forge lalsuite` 
- `conda install -c conda-forge pesummary`

   
#### Documentation
- The formalism of the LALInference package is described in the following article: 
[Phys. Rev. D 91, 042003 (2015)](https://arxiv.org/abs/1409.7215)
- A lecture of GW parameter estimation with Bayesian inference: [Recording of GWOSC ODW #2](https://www.youtube.com/watch?v=_NaAKeVJe1s&t=1s)
- All the GWOSC open data workshops lectures: [Link to GWOSC ODW](https://www.gw-openscience.org/workshops/)

#### Acknowledgements
This tutorial is created by Leïla Haegel (leila.haegel@ligo.org), based on utilitary software designed by the LVK collaboration members. A special thanks is due to Charlie Hoy and Marc Arène for their assistance in understanding `PESummary` and `LALInference`.

**Last update**: May 2022


##  A bit of theory

**Introduction**: `LALInference` [1] uses Markov chains to sample the posterior probability of the GW source parameters (for GW emitted by binary systems of black holes and/or neutron stars). 

**Markov chains**: Markov chains are semi-random walks in specific space, some Markov chain algorithms such as Metropolis-Hastings or Nested Sampling aim at having the semi-random walk steps be proportonal to a target distribution. In LALInference the target distribution is the joint posterior probability of the GW source parameters (such as mass, spin, binary localisation and orientation. 

**Posterior probability**: The posterior probability is given by the Bayes theorem: 

$$ p(\theta | d, H) = \frac{p(d|\theta,H) \ p(\theta|H)}{p(d|H)} $$

where $d$ are the data, $\theta$ are the GW parameters, and $H$ the hypotheses according to which the posterior probability is computed. $p(\theta | d, H)$ is the posterior probability, $p(\theta|H)$ is the prior probability, $p(d|H)$ is the evidence, and $p(d|\theta,H)$ is the likelihood given by [1]:

$$ p(d|\vec{\theta},H_S,S_n(f),\vec{\eta}) = exp \sum_i \left[ -\frac{2 |\tilde{h_i}(\vec{\theta}) - \tilde{d_i}|^2}{T \eta(f_i) S_n(f_i)} - \frac{1}{2} log(\pi \eta_i T S_n(f_i)/2 \right] $$

where $p(d|\vec{\theta},H_S,S_n(f),\vec{\eta})$ is the likelihood marginalised over the calibration envelops, $d_i$ is the strain and $S_n(f_i)$ is the Power Spectral Density (PSD). We therefore need all those quantities, that are described in details below. 

**LALInference**: `LALInference` is a software library computing the posterior probability $p(\theta | d, H)$ with a Markov chain process. It returns the steps of the chains, i.e. the value of the posterior probability at different points of the parameter space. Their distributions enable to infer the most probable value and credible intervals of the GW source parameters. This tutorial downloads the necessary inputs and configures a `LALInference` run.


[1] [Phys. Rev. D 91, 042003 (2015)](https://arxiv.org/abs/1409.7215)


##  Installation 

In [1]:
# -- Those packages will be needed for the execution
#! pip install -q lalsuite pesummary

**Important:** With Google Colab, you may need to restart the runtime after running the cell above.

## Initialization

We begin by importing some commonly used functions

In [2]:
import os 
import lal
import json
import numpy as np
from pesummary.gw import fetch
from pesummary.io import read
from pesummary.gw.file.calibration import CalibrationDict

Then we create the directory in which we will work for this tutorial. All the input and output files will be stored in `tuto3.3` (modify at your convenience if needed). 

In [3]:
# get the local workdir directory name
cwd = os.getcwd()

# create a new directory called tuto3.3 for the run
workdir = cwd + '/tuto_A.1/'
os.makedirs(workdir, exist_ok=True)

## Getting the data: GW190828_063405

In this notebook, we analyse GW190828_063405. Information on the event can be found on the [GWOSC page](https://www.gw-openscience.org/eventapi/html/GWTC-2/GW190828_063405/). 

The notebook can be adapted to analyse any other event from the [GWTC-2](https://www.gw-openscience.org/GWTC-2/), [2.1](https://www.gw-openscience.org/GWTC-2.1/) & [3](https://www.gw-openscience.org/GWTC-3/) catalogs. 



In [4]:
evt = 'GW190828_063405'
catalog = 'GWTC-2'


### - PESummary files

The PE files contains the PSD, calibration envelops and inference run configuration options, as well as the posterior samples from the `LALInference` runs performed by the LVK collaboration.

The file is retrieved and parsed using the `pesummary` package (more information on the [PESummary documentation](https://lscsoft.docs.ligo.org/pesummary/unstable_docs/index.html)).





We start by downloading the PESummary files in the working directory (they will be saved in a directory named as the event): 



In [5]:
fetch.fetch_open_samples(evt, catalog=catalog, 
                         unpack=True, read_file=False, delete_on_exit=False, 
                         outdir=workdir)


Extracting all files from /tmp/astropy-download-505085-9k8b2xzt


PosixPath('/home/leila.haegel/open_data/odw_2022/odw-2022/Tutorials/Advanced_topics/tuto_A.1/GW190828_063405')

Then we open it, after checking that it is downloaded as expected: 

In [6]:
pe_path = workdir + evt + '/' + evt + '.h5'

if os.path.exists(pe_path): 
    print('Parameter estimation file found in: ' + pe_path + '\nOpening it.')
    pe_data = read(pe_path, package="gw")
else: 
    print('Error: parameter estimation file not found in: ' + pe_path + 
          '\n You need to download the PE file manually from the GWOSC: ' + 
          'https://www.gw-openscience.org/eventapi/html/'+ catalog + '/' + evt)

Parameter estimation file found in: /home/leila.haegel/open_data/odw_2022/odw-2022/Tutorials/Advanced_topics/tuto_A.1/GW190828_063405/GW190828_063405.h5
Opening it.


/home/leila.haegel/.conda/envs/lalsuite7.5/lib/python3.9/site-packages/pesummary/utils/dict.py:153: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  self.samples = np.array(_samples)


Several analyses are included in the PESummary file. 
We will chose the analysis: 
- starting with `C01` (release-level calibration)  
- using the `IMRPhenomPv2` approximant. The approximant refers to the model used to simulate the gravitational waveform for the analysis. It does not impact the extraction of the PSD nor the calibration file, but will impact the options chosen to run the parameter estimation later in the tutorial. 

In [7]:
label = 'C01:IMRPhenomPv2'

if label in pe_data.labels: 
    print(label + ' found in the list of analyses. Analyses available are: ', pe_data.labels)
else: 
    print(label + ' not in the list of analyses. Analyses available are: ', pe_data.labels)



C01:IMRPhenomPv2 found in the list of analyses. Analyses available are:  ['C01:IMRPhenomD', 'C01:IMRPhenomPv2', 'C01:NRSur7dq4', 'C01:SEOBNRv4P', 'C01:SEOBNRv4PHM', 'PrecessingSpinIMR', 'PrecessingSpinIMRHM', 'PublicationSamples', 'ZeroSpinIMR']


### - PSD files

The Power Spectral Density (PSD) is the distribution of the signal power in the frequency domain. The PSD can be obtained from the PE samples file using `pesummary`. The PSD must be downloaded with 1 file / interferometer as done below (any directory can be specfied, working directory used in this example). 



In [8]:
psds = pe_data.psd[label]
ifos = psds.detectors

psd_files = {}

for ifo in ifos: 
    freq = psds[ifo].frequencies
    psd  = psds[ifo].strains
    
    psd_path = workdir + evt + '_PSDs_' + ifo + '.dat'
    psd_files[ifo] = psd_path

    np.savetxt(psd_path , np.column_stack([freq, psd]), delimiter='\t')
    
    print('PSD file downloaded in: ', psd_path)



PSD file downloaded in:  /home/leila.haegel/open_data/odw_2022/odw-2022/Tutorials/Advanced_topics/tuto_A.1/GW190828_063405_PSDs_H1.dat
PSD file downloaded in:  /home/leila.haegel/open_data/odw_2022/odw-2022/Tutorials/Advanced_topics/tuto_A.1/GW190828_063405_PSDs_L1.dat
PSD file downloaded in:  /home/leila.haegel/open_data/odw_2022/odw-2022/Tutorials/Advanced_topics/tuto_A.1/GW190828_063405_PSDs_V1.dat


### - Calibration envelops files

The calibration envelops represent the uncertainty due to the detector calibration. The files can be obtained from the PE samples file using `pesummary`. They must be downloaded with 1 file / interferometer as done below (any directory can be specfied, working directory used in this example). 

In [9]:
calib_envs = pe_data.priors['calibration'][label]
calib_data = CalibrationDict(calib_envs)
ifos = calib_data.detectors

calib_files = {}

for ifo in ifos: 

    freq = calib_data[ifo].frequencies
    mag  = calib_data[ifo].magnitude
    phi  = calib_data[ifo].phase
    mag_m1s = calib_data[ifo].magnitude_lower
    phi_m1s = calib_data[ifo].phase_lower
    mag_p1s = calib_data[ifo].magnitude_upper
    phi_p1s = calib_data[ifo].phase_upper
    
    calib_path = workdir + evt + '_CalEnv_' + ifo + '.txt'
    calib_files[ifo] = calib_path
    col_title = 'Frequency Median Mag Phase (Rad) -1 Sigma Mag -1 Sigma Phase +1 Sigma Mag +1 Sigma Phase'
    np.savetxt(calib_path, np.column_stack([freq, mag, phi, mag_m1s, phi_m1s, mag_p1s, phi_p1s]), delimiter=' ', header=col_title)
    
    print('Calibration file downloaded in: ', calib_path)



Calibration file downloaded in:  /home/leila.haegel/open_data/odw_2022/odw-2022/Tutorials/Advanced_topics/tuto_A.1/GW190828_063405_CalEnv_H1.txt
Calibration file downloaded in:  /home/leila.haegel/open_data/odw_2022/odw-2022/Tutorials/Advanced_topics/tuto_A.1/GW190828_063405_CalEnv_L1.txt
Calibration file downloaded in:  /home/leila.haegel/open_data/odw_2022/odw-2022/Tutorials/Advanced_topics/tuto_A.1/GW190828_063405_CalEnv_V1.txt


### - Strain files

The strain files are available on the GWOSC website and can be downloaded below using the `pesummary` package. 
The functions options refer to the following signal properties: 

**Duration**: Up to GWTC-3, all the gravitational waves detected from binary black hole systems have signal duration < 32 s.
For binary systems containing one or two neutron stars, the signal is longer and strain files of 4096 s must be used. 

**Sampling rate**: The sampling rate can be determined by approximating that the maximal frequency $f_{max}$ to the frequency at the innermost stable circular orbit $f_{ISCO}$:  $ f_{max} \simeq f_{ISCO}  \simeq \frac{4.4 kHz}{M_{tot}} $

For a total mass $ M_{tot} > 4$ for binary black holes, we find the higher bound on the maximal frequency: 
$ f_{max} \simeq 1048 Hz $

The sampling rate corresponds to the Nyquist frequency: 
$f_{Nyquist} = 2 f_{max}  = 2048 Hz$

so the files with a sampling rate of $ f_{Nyquist} = 4096 Hz$ can be used for any binary system of black holes detected by the LVK.



In [10]:
strain_files = {}

for ifo in ifos: 
    
    strain = fetch.fetch_open_strain(evt, IFO=ifo, duration=32, sampling_rate=4096., format='gwf', 
                                     delete_on_exit=True, read_file=False, outdir=workdir)
    strain_files[ifo] = str(strain)
 

The strain must be passed to `LALInference` using cache files, created with a bash command: 

In [11]:
strain_cache_files = {}

for ifo in ifos: 
    strain_cache_files[ifo] = strain_files[ifo].split('.gwf')[0]+ '.lcf'
    command = 'ls ' + strain_files[ifo] +' | lalapps_path2cache > ' + strain_cache_files[ifo] 
    print(command)
    os.system(command)

ls /home/leila.haegel/open_data/odw_2022/odw-2022/Tutorials/Advanced_topics/tuto_A.1/H-H1_GWOSC_4KHZ_R1-1251009248-32.gwf | lalapps_path2cache > /home/leila.haegel/open_data/odw_2022/odw-2022/Tutorials/Advanced_topics/tuto_A.1/H-H1_GWOSC_4KHZ_R1-1251009248-32.lcf
ls /home/leila.haegel/open_data/odw_2022/odw-2022/Tutorials/Advanced_topics/tuto_A.1/L-L1_GWOSC_4KHZ_R1-1251009248-32.gwf | lalapps_path2cache > /home/leila.haegel/open_data/odw_2022/odw-2022/Tutorials/Advanced_topics/tuto_A.1/L-L1_GWOSC_4KHZ_R1-1251009248-32.lcf
ls /home/leila.haegel/open_data/odw_2022/odw-2022/Tutorials/Advanced_topics/tuto_A.1/V-V1_GWOSC_4KHZ_R1-1251009248-32.gwf | lalapps_path2cache > /home/leila.haegel/open_data/odw_2022/odw-2022/Tutorials/Advanced_topics/tuto_A.1/V-V1_GWOSC_4KHZ_R1-1251009248-32.lcf


## Chose the LALInference options

The `LALInference` software performs parameter estimation with Bayesian inference, using Markov chain techniques. Two types of Markov chaim algorithms are implemented : Markov Chain Monte-Carlo (MCMC) and nested sampling. `LALInference` supports a wide range of options, from the input files specification to the prior to set up to the number of parallel inference runs to launch. All those options are stored in the config section of the `PESummary` files. In the notebook, we read the  `PESummary` file to create a bash file that will start the inference on our machine with the desired options.   

We will use the information in the sections belows to set up our bash file:

In [12]:
pe_data.config[label].keys()

dict_keys(['analysis', 'bayeswave', 'condor', 'data', 'datafind', 'engine', 'input', 'lalinference', 'ligo-skymap-from-samples', 'ligo-skymap-plot', 'mpi', 'paths', 'resultspage', 'skyarea', 'statevector'])

### - Create the bash file

We know create a bash file that we will write with the necessary options. 

In [13]:
run_path = workdir + 'lalinference_run.sh'
run_path

'/home/leila.haegel/open_data/odw_2022/odw-2022/Tutorials/Advanced_topics/tuto_A.1/lalinference_run.sh'

### - Specify the inference executable

First specify the `conda` environment where LAL is installed. The path needs to be modified according to your machine, specifying the link to the virtual environment. 

In [14]:
env_path = '/cvmfs/oasis.opensciencegrid.org/ligo/sw/conda/envs/igwn-py38/'

Then we chose the executable and relevent options from the `analysis` and `engine` sections of the config options.

**For nested sampling**, we need to specify: 
- the number of live points `nlive` 
- the number of points `maxmcmc` for independent MCMC chains (optional)
- the tolerance on the evidence `tolerance` to obtain to stop the inference (optional)


In [15]:
if pe_data.config[label]['analysis']['engine'] == 'lalinferencenest': 
    with open(run_path, "w") as run_file:
        print(env_path + 'bin/lalinference_nest \\', file=run_file)
        print('--nlive ' + pe_data.config[label]['engine']['nlive'] + ' \\', file=run_file)
        if 'maxmcmc' in pe_data.config[label]['engine']: 
            print('--maxmcmc ' + pe_data.config[label]['engine']['maxmcmc'] + ' \\', file=run_file)
        if 'tolerance' in pe_data.config[label]['engine']: 
            print('--tolerance ' + pe_data.config[label]['engine']['tolerance'] + ' \\', file=run_file)



**For MCMC**, we need to specify: 
- the path to the MPI executable for parallelisation
- the number of parallel jobs `np`
- the number of independent samples `neff` to obtain to stop the inference (optional)
- the number of temperature chains `ntemps` (optional)
- the option to adapt the temperature chains `adapt-temps` (optional)


In [16]:
if pe_data.config[label]['analysis']['engine'] == 'lalinferencemcmc': 
    with open(run_path, "w") as run_file:
        print(env_path + 'bin/lalinference_mpi_wrapper \\', file=run_file)
        print('--mpirun ' + env_path + 'bin/mpirun \\', file=run_file)
        print('--executable ' + env_path + 'bin/lalinference_mcmc \\', file=run_file)
        print('--np ' + pe_data.config[label]['engine']['np'] + ' \\', file=run_file)

        if 'neff' in pe_data.config[label]['engine']: 
            print('--neff ' + pe_data.config[label]['engine']['neff'] + ' \\', file=run_file)
        if 'ntemps' in pe_data.config[label]['engine']: 
            print('--ntemps ' + pe_data.config[label]['engine']['ntemps'] + ' \\', file=run_file)
        if 'adapt-temps' in pe_data.config[label]['engine']: 
            print('--adapt-temps \\', file=run_file)


### - Specify the input files

Specify the location of the input files for each interferometer, with the initial frequency extracted from the `lalinference` section of the configuration options.


In [17]:
if 'H1' in ifos: 
    with open(run_path, "a") as run_file:
        print('--ifo H1 \\', file=run_file)
        print('--H1-cache ' + strain_cache_files['H1'] + ' \\', file=run_file)
        print('--H1-channel H1:GWOSC-4KHZ_R1_STRAIN \\', file=run_file)
        if 'flow' in pe_data.config[label]['lalinference']: 
            f_low = pe_data.config[label]['lalinference']['flow'].replace('\'', '\"')
            print('--H1-flow ' + str(json.loads(f_low)['H1']) + ' \\', file=run_file)
        print('--H1-psd ' + psd_files['H1'] + ' \\', file=run_file)
        print('--H1-spcal-envelope ' + calib_files['H1'] + ' \\', file=run_file)


In [18]:
if 'L1' in ifos: 
    with open(run_path, "a") as run_file:
        print('--ifo L1 \\', file=run_file)
        print('--L1-cache ' + strain_cache_files['L1'] + ' \\', file=run_file)
        print('--L1-channel L1:GWOSC-4KHZ_R1_STRAIN \\', file=run_file)
        if 'flow' in pe_data.config[label]['lalinference']: 
            f_low = pe_data.config[label]['lalinference']['flow'].replace('\'', '\"')
            print('--L1-flow ' + str(json.loads(f_low)['L1']) + ' \\', file=run_file)
        print('--L1-psd ' + psd_files['L1'] + ' \\', file=run_file)
        print('--L1-spcal-envelope ' + calib_files['L1'] + ' \\', file=run_file)


In [19]:
if 'V1' in ifos: 
    with open(run_path, "a") as run_file:
        print('--ifo V1 \\', file=run_file)
        print('--V1-cache ' + strain_cache_files['V1'] + ' \\', file=run_file)
        print('--V1-channel V1:GWOSC-4KHZ_R1_STRAIN \\', file=run_file)
        if 'flow' in pe_data.config[label]['lalinference']: 
            f_low = pe_data.config[label]['lalinference']['flow'].replace('\'', '\"')
            print('--V1-flow ' + str(json.loads(f_low)['V1']) + ' \\', file=run_file)
        print('--V1-psd ' + psd_files['V1'] + ' \\', file=run_file)
        print('--V1-spcal-envelope ' + calib_files['V1'] + ' \\', file=run_file)


### - Specify the calibration 

Chose if the calibration envelops are sampled during the run, and specify the number of nodes from the `engine` section of the configuration options.

In [20]:
if 'enable-spline-calibration' in pe_data.config[label]['engine']: 
    with open(run_path, "a") as run_file:
        print('--enable-spline-calibration \\', file=run_file)
        
if 'spcal-nodes' in pe_data.config[label]['engine']: 
    with open(run_path, "a") as run_file:
        print('--spcal-nodes ' + pe_data.config[label]['engine']['spcal-nodes'] + ' \\', file=run_file)
        

### - Specify the signal options

The options are extracted from the `engine` section of the configuration options, including: 
- the length of the data segment to analyse `seglen` (in seconds)
- the sampling rate `srate` (in Herts)
- the waveform approximant `approx`
- the reference frequency `fref` (optional)

The trigger time `trigtime` that must be copied from the [GWOSC page](https://www.gw-openscience.org/eventapi/html/GWTC-2/GW190828_063405/). 

In [21]:
trig_time = 1251009263.8

In [22]:
with open(run_path, "a") as run_file:
    print('--trigtime ' + str(trig_time) + ' \\', file=run_file)
    print('--srate ' + pe_data.config[label]['engine']['srate'] + ' \\', file=run_file)
    print('--seglen ' + pe_data.config[label]['engine']['seglen'] + ' \\', file=run_file)
    print('--approx ' + pe_data.config[label]['engine']['approx'] + ' \\', file=run_file)

if 'fref' in pe_data.config[label]['engine']: 
    with open(run_path, "a") as run_file:
        print('--fref ' + pe_data.config[label]['engine']['fref'] + ' \\', file=run_file)


### - Specify the priors

The priors are specified in the `engine` section of the configuration options, with the bounds of syntax: `[param]-min` and `[param]-max`.

In [23]:
for key in pe_data.config[label]['engine']:
    if '-min'in key: 
        with open(run_path, "a") as run_file:
            print('--' + key + ' ' + pe_data.config[label]['engine'][key] + ' \\', file=run_file)
    if '-max'in key: 
        with open(run_path, "a") as run_file:
            print('--' + key + ' ' + pe_data.config[label]['engine'][key] + ' \\', file=run_file)



### - Specify the output options

The posterior samples will be saved in `lalinference_samples.hdf5` and we will specify options to resume the inference if interupted and to print the progress into `lalinference.out` and `lalinference.err` files.

In [24]:
posterior_path = workdir + 'lalinference_samples.hdf5'
out_path = workdir + 'lalinference.out'
err_path = workdir + 'lalinference.err'


In [25]:
with open(run_path, "a") as run_file:
    print('--outfile ' + posterior_path + ' \\', file=run_file)
    print('--resume \\', file=run_file)
    print('--progress \\', file=run_file)
    print('> ' + out_path + ' 2> ' + err_path + ' &', file=run_file, end="")


## Start the inference

We will now execute the bash file with a bash command. It is executed from the notebook, but can be started from the terminal as well (in the working directory). The inference process is long and often requires several days to be completed. 

In [26]:
command = 'bash ' + run_path

print('Command: ' + command)

os.system(command)

Command: bash /home/leila.haegel/open_data/odw_2022/odw-2022/Tutorials/Advanced_topics/tuto_A.1/lalinference_run.sh


0